In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

<p style="color:purple; font-weight:bold">
[OA-ReactDiff] is the first diffusion-based generative model for generating  <U>3D chemical reactions</U>, which not only accelerates the search for 3D structures in chemical reactions by a factor of 1000, but also generates and explores <U>new and unknown</U> chemical reactions.
</p>

After completing this tutorial, you will be able to:

* Learn some of the applications and potential difficulties of diffusion model in chemistry and materials

* Understand the principles of running [OA-ReactDiff] and apply them to your projects

**It will take [at most] 20 minutes to read and run through the tutorial, so let's get started!**

Link to article 👉 [OA-ReactDiff](https://arxiv.org/abs/2304.06174)

<p style="color:blue; font-weight:bold">
If you have any ideas about applying OA-ReactDiff to your own work, but are not sure if it's entirely appropriate or you don't know how to get started, please feel free to contact the author, Chenru Duan, via email: duanchenru@gmail.com
</p>

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/4af440e9b48747388fbd9ec9533b482a/mGNnjUDyymMo9ML3xOiF1Q.png" alt="cover_art" width="500" title="cover_art">
</div>

# Table of Content

* [Introduction to the Denoising Diffusion Probabilistic Models](#ddpm)
* [Generating a molecule](#single-molecule)
* [How about generating a chemical reaction?](#reaction)
* [OA-ReactDiff usage scenarios](#application)
    * [1. No preprocessing required, generation is so simple](#no-preprocessing)
    * [2. The gift of stochastic processes to compensate for chemical intuition.](#stochasticity)
    * [3. Give me some atoms and I'll generate all the reactions.](#exploration)
* [The future lies in generative AI](#outlook)


# A brief introduction to Denoising Diffusion Probabilistic Models (DDPM) <a id ='ddpm'></a>

DDPM is a diffusion model based on probabilistic statistical generation and random noise generation of data. It transforms noisy data into clean data samples by modeling a diffusion process.

These models focus on modeling the distribution of the data and simulate a gradual evolution from a simple distribution to a more complex distribution. The basic concept of a diffusion model is to transform a simple and easily sampled distribution, usually a normal distribution, into a more complex data distribution. This transformation is achieved through a series of reversible operations. Once the model has learned the transformation process, it can generate new samples by starting from a point in the simple distribution and gradually "denoising" to the desired complex data distribution.

The training of the DDPM model requires knowledge of the parameters of the diffusion process to effectively capture the relationship between clean and noisy data at each transformation step. In the generation process, DDPM starts with noisy data (e.g., noisy images) and iteratively applies the learned transformations in reverse to obtain denoised and realistic data samples.

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/bcae8391a60845f990e6387bb4ac3977/ox_I-p5BPAeFzFg3Ojh8eQ.png" alt="image_diffusion" width="850" title="image_diffusion">
    <p style='font-size:0.8rem; font-weight:bold'>Figure 1｜Diffusion generation model in generating images</p>
</div>

Supplemental Reading 🔽

[An Introduction to Diffusion Models for Machine Learning](https://encord.com/blog/diffusion-models/#:~:text=Diffusion%20models%20are%20a%20class%20of%20generative%20models%20that%20simulate,a%20sequence%20of%20invertible%20operations.)

[Diffusion Models: A Comprehensive Survey of Methods and Applications](https://arxiv.org/pdf/2209.00796.pdf)

For those who want to play with DDPM image generation and read Chinese, please refer to this notebook tutorial: [Diffusion Models: You still don't understand the basics of diffusion modeling?](https://nb.bohrium.dp.tech/detail/2412844875)

# Tired with images, how about generating molecules?<a id ='single-molecule'></a>

Recently, the framework of DDPM has been progressively applied to tasks related to chemical molecules. Examples include generating [3D structures of organic small molecules](https://arxiv.org/abs/2203.17003), [protein-small molecule docking](https://arxiv.org/abs/2210.01776), and [protein structure-based drug design](https://arxiv.org/abs/2210.13695).

The core difference between the generation of molecules and the image generation above is their <span style="color: orange;">**representation and symmetry**</span>:

* <span style="color: orange;">*Representation*</span>: for a color image of NxN pixels, we generally represent it as a (N, N, 3) tensor, where the last 3 denotes the three primary RGB colors. And for a molecule, we need to use (atomic categories, coordinate information) to represent each atom.

* <span style="color: orange;">*Symmetry*</span>: In images, we generally do not require strict symmetry. It is common to use augmentation to make the model "remember" that the object is not distorted with respect to the 2D rotation. But in molecules, because we learn 3D objects directly and require high accuracy, augmentation is often too expensive and not accurate enough. Thus, we need to embed symmetry into the model.

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/bcae8391a60845f990e6387bb4ac3977/1FzbkAcENo9bPWbm1xJHOQ.png", alt="mol_diffusion" width="850" title="mol_diffusion">
    <p style='font-size:0.8rem; font-weight:bold'>Figure 2｜Diffusion Generation Modeling of a Single Molecule</p>
</div>

## SE(3) symmetry is indispensable for generating molecular time models

While we are generating 3D molecules, the symmetry of the physics on the molecule is needed to be maintained. One way to do this is to use graph network models with SE(3) symmetry. A simple way to understand this is that for a molecule, we get the same result by rotating the molecule (D(g)) and then using the model prediction (f) as we do by using the model prediction and then rotating it, i.e., $D(g) \circ f = f \circ D'(g)$
 
A previous notebook written by Siyuan has a [detailed introduction to SE(3) symmetry](https://nb.bohrium.dp.tech/detail/9619364424). Those who would like to dig deeper can supplement their studies 📚. We will only give one example here for visualization experience.

In [2]:
# --- Importing and defining some functions ----
import torch
import py3Dmol
import numpy as np

from typing import Optional
from torch import tensor
from e3nn import o3
from torch_scatter import scatter_mean

from oa_reactdiff.model import LEFTNet

default_float = torch.float64
torch.set_default_dtype(default_float)  # Use double precision for more accurate testing


def remove_mean_batch(
    x: tensor, 
    indices: Optional[tensor] = None
) -> tensor:
    """Remove the mean from each batch in x

    Args:
        x (tensor): input tensor.
        indices (Optional[tensor], optional): batch indices. Defaults to None.

    Returns:
        tensor: output tensor with batch mean as 0.
    """
    if indices == None:
         return x - torch.mean(x, dim=0)
    mean = scatter_mean(x, indices, dim=0)
    x = x - mean[indices]
    return x


def draw_in_3dmol(mol: str, fmt: str = "xyz") -> py3Dmol.view:
    """Draw the molecule

    Args:
        mol (str): str content of molecule.
        fmt (str, optional): format. Defaults to "xyz".

    Returns:
        py3Dmol.view: output viewer
    """
    viewer = py3Dmol.view(1024, 576)
    viewer.addModel(mol, fmt)
    viewer.setStyle({'stick': {}, "sphere": {"radius": 0.36}})
    viewer.zoomTo()
    return viewer


def assemble_xyz(z: list, pos: tensor) -> str:
    """Assembling atomic numbers and positions into xyz format

    Args:
        z (list): chemical elements
        pos (tensor): 3D coordinates

    Returns:
        str: xyz string
    """
    natoms =len(z)
    xyz = f"{natoms}\n\n"
    for _z, _pos in zip(z, pos.numpy()):
        xyz += f"{_z}\t" + "\t".join([str(x) for x in _pos]) + "\n"
    return xyz

### Building a LEFTNet model

A simple test is performed to verify SE(3) symmetry. The model here is for testing, so we only need to build a very small model.

Note: [LEFTNet](https://arxiv.org/abs/2304.04757) is a new SOTA-level SE(3) graph neural network. Although we use LEFTNet here, the properties it exhibits are model-independent (other SE(3) models, such as [EGNN](https://arxiv.org/pdf/2102.09844.pdf), will give the same results)

TL: EGNN is not SE$(3)$  equivariant?

In [3]:
num_layers = 2
hidden_channels = 8
in_hidden_channels = 4
num_radial = 4

model =  LEFTNet(
    num_layers=num_layers,
    hidden_channels=hidden_channels,
    in_hidden_channels=in_hidden_channels,
    num_radial=num_radial,
    object_aware=False,
)

sum(p.numel() for p in model.parameters() if p.requires_grad)

/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


7882


### Creating a water molecule as a test system

In [4]:
h = torch.rand(3, in_hidden_channels)
z = ["O", "H", "H"]
pos = tensor([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
]).double()  # For convenience, we'll set the H-O-H angle here to 90 degrees
edge_index = tensor([
    [0, 0, 1, 1, 2, 2],
    [1, 2, 0, 2, 0, 1]
]).long()  # Using the fully-connected approach, here the edges are undirected

In [5]:
h

tensor([[0.1913, 0.7247, 0.2491, 0.3370],
        [0.8771, 0.7150, 0.3536, 0.8162],
        [0.4892, 0.7453, 0.8509, 0.2172]])

In [6]:
xyz = assemble_xyz(z, pos)
view = draw_in_3dmol(xyz, "xyz")

view  # Display Molecules

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Forward prediction of a scalar and vector using LEFTNet

In [7]:
_h, _pos, __ = model.forward(
    h=h,
    pos=remove_mean_batch(pos),
    edge_index=edge_index,
)  # _h is the output feature, _pos is the output position, the former is a scalar (TL !?), the latter is a vector

In [8]:
_h

tensor([[-0.6965, -0.8421,  0.0524, -1.0898],
        [-0.8283, -0.9152,  0.1776, -1.0493],
        [-0.6482, -0.8631,  0.1487, -1.1045]], grad_fn=<AddmmBackward0>)

In [9]:
_pos

tensor([[-0.3333, -0.3337,  0.0000],
        [ 0.6674, -0.3332,  0.0000],
        [-0.3329,  0.6671,  0.0000]], grad_fn=<AddBackward0>)

### Random rotation of our water molecule and run it through LEFTNet again

In [10]:
rot = o3.rand_matrix()
pos_rot = torch.matmul(pos, rot).double()

_h_rot, _pos_rot, __ = model.forward(
    h=h,
    pos=remove_mean_batch(pos_rot),
    edge_index=edge_index,
)

### It doesn't matter if it's a scalar or a vector, the result is always equivariant.

In [11]:
torch.max(
    torch.abs(
        _h - _h_rot
    )
)  # The h should remain the same after rotation

tensor(1.5190e-12, grad_fn=<MaxBackward1>)

In [12]:
torch.max(
    torch.abs(
        torch.matmul(_pos, rot).double() - _pos_rot
    )
)  # The rotated pos should rotate

tensor(8.7708e-15, grad_fn=<MaxBackward1>)

# The core of chemistry are reactions from the age of alchemy. How about direct generation of chemical reactions?<a id ='reaction'></a>

Since the ancient days of alchemy, chemistry has been a discipline of <span style="color: orange;">**studying and controlling how substances react with each other**</span>. Now that we have succeeded in generating a single molecule, why don't we extend the current diffusion generation model to <span style="color: orange;">**generating a group of molecules**</span>, such as the "reactants, transition states, generators" that is of interest to everyone in chemistry?

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/cc00542f53e6459e90e866f6c009e25e/2voIGX7BDbS6pZAOIqRohw.png", alt="reaction_diffusion" width="350" title="reaction_diffusion">
    <p style='font-size:0.8rem; font-weight:bold'>Figure 3: Illustration of a chemical reaction. The black line is the energy curve, "reactants, products" corresponds to the poles, and "transition state" corresponds to the saddle point.</p>
</div>


With this in mind, we generalize diffusion generation to multi-system generation. For training, we simultaneously add noise to "reactants (blue), transition states (yellow), and products (red)", and use a graph neural network model with special symmetry (described in detail later) to predict the three added noises. Two models were developed for generation.

1. <span style="color: orange;">*Random generation*</span>. In this case, we generate a new chemical reaction directly from the three Gaussian distributions, i.e., the **new "unknown"** "reactant, transition state, product".
2. <span style="color: orange;">*Conditional Generation*</span>. In chemical reactions, we usually know part of the information. Here we can use inpainting for conditional generation, e.g. given reactants and products, we can directly generate transition states. In this process, we add fixed noise to "reactants, products" at each step and combine them with the transition states predicted by the graph neural network model. Step by step, we iterate until we generate "reactants ("known"), transition states ("unknown"), generators ("known")".
<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/bcae8391a60845f990e6387bb4ac3977/OnS4d6hjIcgEaIZcsyKxbQ.png", alt="reaction_diffusion" width="850" title="reaction_diffusion">>
    <p style='font-size:0.8rem; font-weight:bold'>Figure 4｜Detailed Process of Chemical Reaction Generated by Diffusion Generation Model</p>
</div>


## When we rotate any molecule of a chemical reaction, does that reaction change?
When generating a single molecule or a large multi-molecular system, we generally only need to satisfy the corresponding shift in the output of the model after a change in the SE(3) of the whole system (e.g., rotation of the whole) (second row of the figure below).

In chemical reactions, however, this symmetry is **not strong enough**. In reactions, we also require **SE(3) symmetry with respect to each single object** (third row of the figure below). A simple example is that when we rotate a reactant, the output of the model should only rotate the reactant and the rest of the reaction should remain unchanged, rather than treating it as a "brand new" reaction.

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/bcae8391a60845f990e6387bb4ac3977/7R-rKlaPFQvpW5rVdQbvIw.png", alt="reaction_symmetry" width="350" title="reaction_symmetry">
    <p style='font-size:0.8rem; font-weight:bold'>Figure 5｜Behavior of chemical reactions under different symmetries (top) after different transformations (left)</p>
</div>

## Failure in system-wide SE(3) for maintaining symmetry of individual objects in reactions


### Consider a hypothetical "electrolysis of water" reaction, H2O -> H2 + O.
Note: This reaction process is *purely fictitious* and is only a simplified model to help us understand the symmetry required in the reaction. Also, we have omitted the transition state structure and only considered the symmetry relationship of 'reactants, products'. The inclusion of transition states does not influence the conclusion.

In [13]:
# TL:
ns = [3, ] + [2, 1]
ns

[3, 2, 1]

In [14]:
ns = [3, ] + [2, 1]  # reactant 3 atoms (H2O), product 2 atoms (H2), 1 atom (O radical)
ntot = np.sum(ns) # TL: total number of objects across all states?
mask = tensor([0, 0, 0, 1, 1, 1])  # To distinguish between reactants and products
z = ["O", "H", "H"] +  ["H", "H", "O"]
pos_react = tensor([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
]).double()  # For convenience, we'll set the H-O-H angle here to 90 degrees
pos_prod = tensor([
    [0, 3, -0.4],
    [0, 3, 0.4],
    [0, -3, 0],
])  # Separating the H2 and O radicals
pos = torch.cat(
    [pos_react, pos_prod],
    dim=0,
)  # 拼接 # TL: 拼接 ~= "concatenate"
h  = torch.rand(ntot, in_hidden_channels)

In [15]:
pos

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000],
        [ 0.0000,  3.0000, -0.4000],
        [ 0.0000,  3.0000,  0.4000],
        [ 0.0000, -3.0000,  0.0000]])

In [16]:
h

tensor([[0.5860, 0.3469, 0.9190, 0.2229],
        [0.1671, 0.5135, 0.3136, 0.8096],
        [0.9904, 0.2575, 0.7585, 0.9721],
        [0.4342, 0.7330, 0.1633, 0.9540],
        [0.1711, 0.0293, 0.3755, 0.4502],
        [0.3509, 0.8997, 0.2304, 0.7495]])

In [17]:
xyz = assemble_xyz(z, pos)
view = draw_in_3dmol(xyz, "xyz")

view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [18]:
xyz

'6\n\nO\t0.0\t0.0\t0.0\nH\t1.0\t0.0\t0.0\nH\t0.0\t1.0\t0.0\nH\t0.0\t3.0\t-0.4\nH\t0.0\t3.0\t0.4\nO\t0.0\t-3.0\t0.0\n'


When using the overall SE(3) for a chemical reaction, all of our atoms need to be connected together to ensure that the reactants and products can 'interact' with each other via the graph neural network. Otherwise, even if we change the reactants, the products won't change, and the problem become worse😉

In [19]:
from oa_reactdiff.tests.model.utils import (
    generate_full_eij, # TL: edge index like the two rows of source and target node IDs before.
    get_cut_graph_mask,
)

edge_index = generate_full_eij(ntot)

In [20]:
edge_index

tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
         4, 5, 5, 5, 5, 5],
        [1, 2, 3, 4, 5, 0, 2, 3, 4, 5, 0, 1, 3, 4, 5, 0, 1, 2, 4, 5, 0, 1, 2, 3,
         5, 0, 1, 2, 3, 4]])


### forward projection

In [21]:
_h, _pos, __ = model.forward(
    h=h,
    pos=remove_mean_batch(pos, mask),
    edge_index=edge_index,
)


### Let's rotate the reactants.

In [22]:
rot = o3.rand_matrix()
pos_react_rot = torch.matmul(pos_react, rot).double()

pos_rot = torch.cat(
    [pos_react_rot, pos_prod],
    dim=0,
)  # Combine rotated H2O and unrotated H2 and O radicals

xyz = assemble_xyz(z, pos_rot)
view = draw_in_3dmol(xyz, "xyz")

view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [23]:
pos_rot

tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.2402,  0.1065, -0.9649],
        [ 0.6294, -0.7397, -0.2383],
        [ 0.0000,  3.0000, -0.4000],
        [ 0.0000,  3.0000,  0.4000],
        [ 0.0000, -3.0000,  0.0000]])


### Predict this reaction again. Note that the chemical substance of these two reactions is the same

In [24]:
_h_rot, _pos_rot, __ = model.forward(
    h=h,
    pos=remove_mean_batch(pos_rot, mask),
    edge_index=edge_index,
)

In [25]:
_pos_rot

tensor([[-1.2968e-01,  2.1089e-01,  4.0170e-01],
        [-3.6985e-01,  3.1769e-01, -5.6379e-01],
        [ 5.0137e-01, -5.3021e-01,  1.6301e-01],
        [-1.1425e-04,  1.9975e+00, -3.9842e-01],
        [ 4.8567e-06,  1.9997e+00,  3.9970e-01],
        [ 3.4310e-05, -3.9928e+00,  4.0416e-05]], grad_fn=<AddBackward0>)

In [26]:
_h_rot

tensor([[-0.5619, -0.8606,  0.0530, -1.2367],
        [-0.5049, -0.7969, -0.0518, -1.2034],
        [-0.7093, -0.9140,  0.1142, -1.1912],
        [-0.1108, -0.5676,  0.0504, -1.0636],
        [-0.0528, -0.5248, -0.0782, -1.0869],
        [ 0.3160, -0.2091,  0.1010, -0.7866]], grad_fn=<AddmmBackward0>)

In [27]:
# TL what am I supposed to see from this? And how?

### But we're losing "equivariance" on both scalar and vector❌

In [28]:
torch.max(
    torch.abs(
        _h - _h_rot
    )
)  # The h should remain the same after rotation

tensor(0.0287, grad_fn=<MaxBackward1>)

In [29]:
_pos_rot_prime = torch.cat(
    [
        torch.matmul(_pos[:3], rot),
        _pos[3:]
    ]
)
torch.max(
    torch.abs(
        _pos_rot_prime  - _pos_rot
    )
)  # The rotated pos should rotate

tensor(0.0013, grad_fn=<MaxBackward1>)


## The need for an <span style="color: orange;">**"object-aware"**</span> SE(3) model in chemical reactions

To address this challenge and extend the framework of generative modeling to chemical reactions, we have developed a **generalized approach**, [OA-ReactDiff](https://arxiv.org/abs/2304.06174), that extends the SE(3) model to an <span style="color: orange;">**"object-aware"**</span> SE(3) model. This approach can be applied to variety of graph neural networks and Transformers.

This part contains more technical details. If you are interested, you can click on the article link above🔗. We directly demonstrate the improved results in this notebook.



### Create an "Object-Aware" LEFTNet

In [30]:
model_oa =  LEFTNet(
    num_layers=num_layers,
    hidden_channels=hidden_channels,
    in_hidden_channels=in_hidden_channels,
    num_radial=num_radial,
    object_aware=True,  # Using the object-aware model
)


We assign different atoms to their respective objects, which are "reactants and products" in this case.

In [31]:
edge_index

tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
         4, 5, 5, 5, 5, 5],
        [1, 2, 3, 4, 5, 0, 2, 3, 4, 5, 0, 1, 3, 4, 5, 0, 1, 2, 4, 5, 0, 1, 2, 3,
         5, 0, 1, 2, 3, 4]])

In [32]:
subgraph_mask = get_cut_graph_mask(edge_index, 3)  # 0-2 is the atomic number of the reactant

In [33]:
subgraph_mask.shape # TL: why this shape?

torch.Size([30, 1])

In [34]:
subgraph_mask

tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1]])


### Repeat the rotation test again


In [35]:
_h, _pos, __ = model_oa.forward(
    h=h,
    pos=remove_mean_batch(pos, mask),
    edge_index=edge_index,
    subgraph_mask=subgraph_mask,
)

In [36]:
rot = o3.rand_matrix()
pos_react_rot = torch.matmul(pos_react, rot).double()

pos_rot = torch.cat(
    [pos_react_rot, pos_prod],
    dim=0,
)

_h_rot, _pos_rot, __ = model_oa.forward(
    h=h,
    pos=remove_mean_batch(pos_rot, mask),
    edge_index=edge_index,
    subgraph_mask=subgraph_mask,
)

### We verified that the "object-aware" LEFTNet will maintain the desired symmetry ✅

In [37]:
torch.max(
    torch.abs(
        _h - _h_rot
    )
)  # The h should remain the same after rotation

tensor(1.1355e-13, grad_fn=<MaxBackward1>)

In [38]:
_pos_rot_prime = torch.cat(
    [
        torch.matmul(_pos[:3], rot),
        _pos[3:]
    ]
)

torch.max(
    torch.abs(
        _pos_rot_prime  - _pos_rot
    )
)  # The rotated pos should rotate

tensor(4.9960e-16, grad_fn=<MaxBackward1>)

# From days to seconds, OA-ReactDiff accelerates the generation of transition state (TS) structures

In experiments, the time scale of the existence of reactants and products is relatively long, making them relatively easy to characterize (e.g. NMR, mass spectrometry, etc.). However, <span style="color: red;">**experimentally characterizing transition state structures is very difficult**</span>. This is because transition states exist for a very short period of time and have higher energy compared to reactants or products, making them difficult to separate and study directly. In terms of calculations, although methods like nudged elastic band have been developed, they are time-consuming and often require running on a machine that contains 24 CPU for a day, with <span style="color: red;">**a very low**</span> success rate (around 30%) 😭.

However, with [the OA-ReactDiff architecture developed in our article](https://arxiv.org/abs/2304.06174), can reduce the search time for transition states **from several days to 6 seconds**, greatly accelerating the exploration of chemical reaction mechanisms, improving success rates, and enabling the study of more complex chemical reaction networks. We have prepared three examples to demonstrate the application scenarios of OA-ReactDiff 🎬.

## Correct symmetry removes the need for all cumbersome pre-processing<a id ='no-preprocessing'></a>
In computation, traditional machine learning methods usually require a one-to-one correspondence between reactants and products, and the geometric positions between each molecule in the reactants also need to be carefully adjusted if there are multiple reactants. These pre-processing steps, not only take time, are practically unfeasible in many unknown reactions.

However, OA-ReactDiff ensures the symmetry required in all chemical reactions, So we **don't need to do any pre-processing for the reactions**.

In [39]:
# --- Importing necessary function ---
from torch.utils.data import DataLoader

from oa_reactdiff.trainer.pl_trainer import DDPMModule


from oa_reactdiff.dataset import ProcessedTS1x
from oa_reactdiff.diffusion._schedule import DiffSchedule, PredefinedNoiseSchedule

from oa_reactdiff.diffusion._normalizer import FEATURE_MAPPING
from oa_reactdiff.analyze.rmsd import batch_rmsd

from oa_reactdiff.utils.sampling_tools import (
    assemble_sample_inputs,
    write_tmp_xyz,
)


### Import the pre-trained model and redefine the schedule.

In [40]:
# TL fix: {
from oa_reactdiff.trainer.pl_trainer import DDPMModule
# } fix. Why didn' this carry over from the previous cell import statement?

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")
print(device) # TL

ddpm_trainer = DDPMModule.load_from_checkpoint(
    checkpoint_path="./pretrained-ts1x-diff.ckpt",
    map_location=device,
)
ddpm_trainer = ddpm_trainer.to(device)

cuda


/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


In [41]:
noise_schedule: str = "polynomial_2"
timesteps: int = 150
precision: float = 1e-5

gamma_module = PredefinedNoiseSchedule(
            noise_schedule=noise_schedule,
            timesteps=timesteps,
            precision=precision,
        )
schedule = DiffSchedule(
    gamma_module=gamma_module,
    norm_values=ddpm_trainer.ddpm.norm_values
)
ddpm_trainer.ddpm.schedule = schedule
ddpm_trainer.ddpm.T = timesteps
ddpm_trainer = ddpm_trainer.to(device)


### Prepare dataset and data loader and select a reaction involving multiple molecules

In [42]:
dataset = ProcessedTS1x(
    npz_path="./oa_reactdiff/data/transition1x/train.pkl",
    center=True,
    pad_fragments=0,
    device=device,
    zero_charge=False,
    remove_h=False,
    single_frag_only=False,
    swapping_react_prod=False,
    use_by_ind=True,
)
loader = DataLoader(
    dataset, 
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=dataset.collate_fn
)
itl = iter(loader)
idx = -1 # TL: why?

len(dataset)

9000

In [43]:
#dataset.transition_state

In [44]:
itl

In [45]:
for i in range(4):  # The 4th sample happens to be a multimolecular reaction
    representations, res = next(itl) #  (reactants, TS, product), label properties
    fragments_nodes = [
    repre["size"] for repre in representations
]
    print(fragments_nodes)
    if i < 4: #== 3:
        print(i)
        print(representations) # TL: how can we see it's multimolecular?
        print("res:")
        print(res)
        print("\n\n") 
idx += 1 # TL: why? not used?
n_samples = representations[0]["size"].size(0)
print(n_samples)
fragments_nodes = [
    repre["size"] for repre in representations
]
conditions = torch.tensor([[0] for _ in range(n_samples)], device=device)

[tensor([7], device='cuda:0'), tensor([7], device='cuda:0'), tensor([7], device='cuda:0')]
0
[{'size': tensor([7], device='cuda:0'), 'pos': tensor([[ 0.2427, -0.8901,  0.2854],
        [-0.5765,  0.0045,  0.8826],
        [-0.6887,  1.1058,  0.2289],
        [ 0.1122,  0.9626, -0.8994],
        [ 0.6299, -0.2113, -0.8175],
        [-1.0335, -0.2771,  1.8197],
        [ 1.3140, -0.6944, -1.4998]], device='cuda:0', dtype=torch.float32), 'one_hot': tensor([[0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], device='cuda:0'), 'charge': tensor([[8],
        [6],
        [7],
        [7],
        [6],
        [1],
        [1]], device='cuda:0', dtype=torch.int32), 'mask': tensor([0, 0, 0, 0, 0, 0, 0], device='cuda:0')}, {'size': tensor([7], device='cuda:0'), 'pos': tensor([[ 0.1031,  0.6314, -1.1980],
        [-0.8480,  0.1328, -0.5836],
        [-0.8011, -0.5178,  0.5847],
   

In [46]:
len(representations)

3

In [47]:
fragments_nodes

[tensor([7], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([7], device='cuda:0')]

In [48]:
# Co-pilot:
# This cell aims to confirm or falsify the interpretation of how `remove_mean_batch`
# handles atomic coordinates from the `representations` obtained from the DataLoader,
# particularly in the context of how OA-ReactDiff's LEFTNet model processes them.

# User's Interpretation to verify:
# 1. The `indices` (or "mask") used with `remove_mean_batch` in the model's core
#    effectively group atoms by reaction stage (reactant, TS, product), leading to
#    independent centering of each stage.
# 2. Within a single stage (e.g., a product stage composed of multiple molecules),
#    `remove_mean_batch` (as used in this context) centers the entire collection of
#    that stage's atoms as one unit. It does not perform further sub-centering
#    for individual molecules within that stage.

print("--- Verifying Interpretation of remove_mean_batch ---")

# Ensure `representations`, `device`, and `remove_mean_batch` are available
# This cell should be run after the cell that defines `remove_mean_batch` (cell 6)
# and after a sample is loaded into `representations` (e.g., cell 46).

# Assuming 'representations' is a list of 3 dicts (reactant, TS, product) for a single sample (batch_size=1)
# Each dict contains 'pos' (atomic positions for that stage) and 'size' (number of atoms in that stage).

stage_names = ["Reactant", "Transition State", "Product"]

for i, stage_repre in enumerate(representations):
    stage_name = stage_names[i]
    pos_stage = stage_repre['pos'] # Positions for the current stage
    num_atoms_in_stage = stage_repre['size'].item()

    print(f"\n--- {stage_name} (Stage {i}) ---")
    print(f"Number of atoms in this stage: {num_atoms_in_stage}")
    # print(f"Original positions (first 2 atoms):\n{pos_stage[:2]}")
    print(f"Mean of original positions for {stage_name}: {torch.mean(pos_stage, dim=0).cpu().numpy()}")

    # In LEFTNet.forward_all_fragments, for each fragment (stage) `i`,
    # `remove_mean_batch(frag_pos[i], frag_indices[i])` is called.
    # `frag_pos[i]` corresponds to `pos_stage`.
    # `frag_indices[i]` corresponds to `node_sample_idx` from collate_fn.
    # For batch_size=1, `node_sample_idx` for all atoms in this stage will be all zeros.
    indices_for_stage_centering = torch.zeros(num_atoms_in_stage, dtype=torch.long, device=pos_stage.device)

    # Apply remove_mean_batch as it would be for this stage's positions
    centered_pos_stage = remove_mean_batch(pos_stage.clone(), indices_for_stage_centering)

    # print(f"Centered positions (first 2 atoms):\n{centered_pos_stage[:2]}")
    print(f"Mean of centered positions for {stage_name}: {torch.mean(centered_pos_stage, dim=0).cpu().numpy()}")

    if stage_name == "Product": # The 4th sample's product stage is multi-molecular
        # We can check the raw data for the 4th sample (index 3)
        # product_num_atoms_per_molecule = dataset.raw_dataset['product']['num_atoms'][3] # 3 is the index for the 4th sample
        # print(f"Structure of product stage (num_atoms per molecule): {product_num_atoms_per_molecule}")
        # This shows that even if product_num_atoms_per_molecule indicates multiple molecules,
        # (e.g. [N1, N2]), the `pos_stage` contains N1+N2 atoms, and they are all centered
        # together by the `remove_mean_batch` call above because `indices_for_stage_centering` is all zeros.
        print(f"Note: If this {stage_name} is multi-molecular (e.g., contains N1+N2 atoms from two molecules),")
        print(f"the above centering operation treated all {num_atoms_in_stage} atoms as a single group.")
        print("This is because `indices_for_stage_centering` was a tensor of all zeros for this stage.")

print("\n--- Conclusion ---")
print("The output above demonstrates that `remove_mean_batch`, when applied in the model's pipeline")
print("for a batch_size of 1, centers each reaction stage (Reactant, TS, Product) independently.")
print("Crucially, if a stage itself is composed of multiple molecules, all atoms of that stage")
print("are centered together as a single unit. The `indices` used in this context do not")
print("facilitate separate centering of individual molecules *within* a single stage.")
print("This aligns with the interpretation that the GNN processes the (potentially multi-molecular)")
print("geometry of an entire stage after that stage as a whole has been centered.")

--- Verifying Interpretation of remove_mean_batch ---

--- Reactant (Stage 0) ---
Number of atoms in this stage: 7
Mean of original positions for Reactant: [ 1.7029899e-08 -2.9802322e-08  2.3416112e-08]
Mean of centered positions for Reactant: [ 1.7029899e-08 -2.7673586e-08  1.7029899e-08]

--- Transition State (Stage 1) ---
Number of atoms in this stage: 7
Mean of original positions for Transition State: [ 3.4059799e-08 -2.1287374e-08  8.5149496e-09]
Mean of centered positions for Transition State: [-8.5149495e-08 -1.7562083e-08  6.3862124e-09]

--- Product (Stage 2) ---
Number of atoms in this stage: 7
Mean of original positions for Product: [ 2.724784e-07 -2.724784e-07  5.449568e-07]
Mean of centered positions for Product: [ 2.724784e-07 -2.724784e-07  5.449568e-07]
Note: If this Product is multi-molecular (e.g., contains N1+N2 atoms from two molecules),
the above centering operation treated all 7 atoms as a single group.
This is because `indices_for_stage_centering` was a tensor of

In [49]:
# TL: I want to see how the 4th set of representations is multi-molecular
from pprint import pprint
for i, repre in enumerate(representations):
    print(stage_names[i])
    #pprint(repre["charge"].sum(dim=0)) # 'size', 'pos', 'one_hot', 'charge', 'mask'
    pprint(repre["mask"].sum(dim=-1))
    print("")
#print([repr.keys() for repr in representations])

# reactants?


Reactant
tensor(0, device='cuda:0')

Transition State
tensor(0, device='cuda:0')

Product
tensor(0, device='cuda:0')



In [50]:
subgraph_mask

tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1]])

In [51]:
subgraph_mask.size()

torch.Size([30, 1])

In [52]:
# TL visualize each state:
atomic_num2sym = {
    1: 'H',    2: 'He',   3: 'Li',   4: 'Be',   5: 'B',    6: 'C',    7: 'N',    8: 'O',    9: 'F',    10: 'Ne',
    11: 'Na',  12: 'Mg',  13: 'Al',  14: 'Si',  15: 'P',   16: 'S',   17: 'Cl',  18: 'Ar',  19: 'K',   20: 'Ca',
    21: 'Sc',  22: 'Ti',  23: 'V',   24: 'Cr',  25: 'Mn',  26: 'Fe',  27: 'Co',  28: 'Ni',  29: 'Cu',  30: 'Zn',
    31: 'Ga',  32: 'Ge',  33: 'As',  34: 'Se',  35: 'Br',  36: 'Kr',  37: 'Rb',  38: 'Sr',  39: 'Y',   40: 'Zr',
    41: 'Nb',  42: 'Mo',  43: 'Tc',  44: 'Ru',  45: 'Rh',  46: 'Pd',  47: 'Ag',  48: 'Cd',  49: 'In',  50: 'Sn',
    51: 'Sb',  52: 'Te',  53: 'I',   54: 'Xe',  55: 'Cs',  56: 'Ba',  57: 'La',  58: 'Ce',  59: 'Pr',  60: 'Nd',
    61: 'Pm',  62: 'Sm',  63: 'Eu',  64: 'Gd', 65: 'Tb',  66: 'Dy',  67: 'Ho',  68: 'Er',  69: 'Tm',  70: 'Yb',
    71: 'Lu',  72: 'Hf',  73: 'Ta',  74: 'W',   75: 'Re',  76: 'Os',  77: 'Ir',  78: 'Pt',  79: 'Au',  80: 'Hg',
    81: 'Tl',  82: 'Pb',  83: 'Bi',  84: 'Po',  85: 'At',  86: 'Rn',  87: 'Fr',  88: 'Ra',  89: 'Ac',  90: 'Th',
    91: 'Pa',  92: 'U',   93: 'Np',  94: 'Pu',  95: 'Am',  96: 'Cm',  97: 'Bk',  98: 'Cf',  99: 'Es', 100: 'Fm',
    101: 'Md', 102: 'No', 103: 'Lr', 104: 'Rf', 105: 'Db', 106: 'Sg', 107: 'Bh', 108: 'Hs', 109: 'Mt', 110: 'Ds',
    111: 'Rg', 112: 'Cn', 113: 'Nh', 114: 'Fl', 115: 'Mc', 116: 'Lv', 117: 'Ts', 118: 'Og'
}

for i, repre in enumerate(representations):
    z = [atomic_num2sym[int(x[0])] for x in repre["charge"]]
    pos = repre["pos"]
    if i == 1: # 0, 1, 2
        break
        
xyz = assemble_xyz(z, pos.cpu())
view = draw_in_3dmol(xyz, "xyz")
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.



The order of atoms in the reactants is intentionally scrambled relative to the products, and two molecules in the products are pulled infinitely far apart (10 A).

In [53]:
new_order_react = torch.randperm(representations[0]["size"].item())
for k in ["pos", "one_hot", "charge"]:
    representations[0][k] = representations[0][k][new_order_react]
    
xh_fixed = [
    torch.cat(
        [repre[feature_type] for feature_type in FEATURE_MAPPING],
        dim=1,
    )
    for repre in representations
]

In [54]:
new_order_react

tensor([4, 0, 1, 2, 3, 6, 5])

In [55]:
xh_fixed

[tensor([[-0.0583, -0.8153, -1.1315,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
           7.0000],
         [ 1.2328, -0.5216, -0.8588,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
           8.0000],
         [ 1.3499,  0.2690,  0.2318,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
           7.0000],
         [ 0.1347,  0.4751,  0.6483,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
           6.0000],
         [-0.7571, -0.2115, -0.2149,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
           6.0000],
         [-1.8343, -0.2778, -0.1928,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           1.0000],
         [-0.0676,  1.0821,  1.5179,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           1.0000]], device='cuda:0', dtype=torch.float32),
 tensor([[ 1.2855,  0.7048, -0.5932,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
           8.0000],
         [ 1.7646, -0.1798,  0.0329,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
           7.0000],
         [-0.4315, -0.8378,  0.6446,  0.00

In [56]:
conditions

tensor([[0]], device='cuda:0')

### Generating TS structures for this set of "reactants, products" without any atomic ordering and fragment arrangement.

In [ ]:
out_samples, out_masks = ddpm_trainer.ddpm.inpaint(
    n_samples=n_samples,
    fragments_nodes=fragments_nodes,
    conditions=conditions,
    return_frames=1,
    resamplings=5,
    jump_length=5,
    timesteps=None,
    xh_fixed=xh_fixed,
    frag_fixed=[0, 2],
)

### Evaluating the quality of generated TS structures



The difference between two molecules with RMSD below 0.1A is almost indistinguishable to the naked eye.

<div>
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/20204/bcae8391a60845f990e6387bb4ac3977/4vUVwvt2hGtpcZwTw296Xg.png", alt="rmsd_visualy" width="850" title="rmsd_visual">
    <p style='font-size:0.8rem; font-weight:bold'>Figure 6 | Structural differences between two molecules at different RMSD values (The C atoms of the two molecules are indicated by blue and yellowish colors, respectively)</p>
</div>

Storing the generated transition state and checking its RMSD against the true transition state for this reaction, we have an RMSD of only 0.02A.



In [ ]:
rmsds = batch_rmsd(
    fragments_nodes, 
    out_samples[0],
    xh_fixed,
    idx=1,
)
write_tmp_xyz(
    fragments_nodes, 
    out_samples[0], 
    idx=[0, 1, 2], 
    localpath="./demo/inpainting"
)

rmsds = [min(1, _x) for _x in rmsds]
[(ii, round(rmsd, 2)) for ii, rmsd in enumerate(rmsds)], np.mean(rmsds), np.median(rmsds)

It can be seen that the atomic numbers of the transition states we generate and the atomic numbers of the reactants do not need to correspond at all, showing that OA-ReactDiff does not require the pre-processing of atomic arrangements

In [ ]:
! cat ./demo/inpainting/gen_0_react.xyz

In [ ]:
! cat ./demo/inpainting/gen_0_ts.xyz


At the same time, although the product consists of multiple molecules, there is no need for a specific geometrical arrangement of them (we've already pulled two molecules to "infinity" before the creation)

In [ ]:
draw_in_3dmol(
    open("./demo/inpainting/gen_0_prod.xyz", "r").read(),
    "xyz"
)


## Surprises for unintented reactions - Gifts from stochastic processes<a id ='stochasticity'></a>

There is a certain amount of randomness in the diffusion generation model due to the intervention of noise, which leads to the fact that given the 3D structure of the 'reactants, products' in time, the transition state structure that comes out of the generation is not the same every time.

So a natural question arises: do these randomly generated structures, apart from the one that is identical to the true transition state, <span style="color: orange;">**have any chemical significance and practical value**</span>❓
    
We will delve into a case study to find the answer to this question.

In [ ]:
from glob import glob
import plotly.express as px

from oa_reactdiff.analyze.rmsd import xyz2pmg, pymatgen_rmsd

from pymatgen.core import Molecule
from collections import OrderedDict


def draw_reaction(react_path: str, idx: int = 0, prefix: str = "gen") -> py3Dmol.view:
    """Draw the {reactants, transition states, products} of the reaction.

    Args:
        react_path (str): path to the reaction.
        idx (int, optional): index for the generated reaction. Defaults to 0.
        prefix (str, optional): prefix for distinguishing true sample and generated structure.
            Defaults to "gen".

    Returns:
        py3Dmol.view: _description_
    """
    with open(f"{react_path}/{prefix}_{idx}_react.xyz", "r") as fo:
        natoms = int(fo.readline()) * 3
    mol = f"{natoms}\n\n"
    for ii, t in enumerate(["react", "ts", "prod"]):
        pmatg_mol = xyz2pmg(f"{react_path}/{prefix}_{idx}_{t}.xyz")
        pmatg_mol_prime = Molecule(
            species=pmatg_mol.atomic_numbers,
            coords=pmatg_mol.cart_coords + 8 * ii,
        )
        mol += "\n".join(pmatg_mol_prime.to(fmt="xyz").split("\n")[2:]) + "\n"
    viewer = py3Dmol.view(1024, 576)
    viewer.addModel(mol, "xyz")
    viewer.setStyle({'stick': {}, "sphere": {"radius": 0.3}})
    viewer.zoomTo()
    return viewer

### Draw our example reaction

The bottom left to top right corresponds to "reactants, transition states, and products".

In [ ]:
draw_reaction("./demo/example-3/ground_truth", prefix="sample")



Take a closer look at the transition state structure.

In [ ]:
draw_in_3dmol(
    open("./demo/example-3/ground_truth/sample_0_ts.xyz", "r").read(),
    "xyz"
)

In [ ]:
#TL reactant
draw_in_3dmol(
    open("./demo/example-3/ground_truth/sample_0_react.xyz", "r").read(),
    "xyz"
)

In [ ]:
#TL product
draw_in_3dmol(
    open("./demo/example-3/ground_truth/sample_0_prod.xyz", "r").read(),
    "xyz"
)



### How many different TSs do we generate for the same reaction?

To save everyone's time, we have gone ahead and used OA-ReactDiff to generate multiple transition state structures for this 'reactant, product', this reaction corresponds to [the example on the right side of Figure 3 in our article](https://arxiv.org/abs/2304.06174).


Sort the generated transition states into categories.

In [ ]:
opt_ts_path = "./demo/example-3/opt_ts/"
opt_ts_xyzs = glob(f"{opt_ts_path}/*ts.opt.xyz")

order_dict = {}
for xyz in opt_ts_xyzs:

    order_dict.update(
        {int(xyz.split("/")[-1].split(".")[0]): xyz}
    )
order_dict = OrderedDict(sorted(order_dict.items()))

opt_ts_xyzs = []
ind_dict = {}
for ii, v in enumerate(order_dict.values()):
    opt_ts_xyzs.append(v)
    ind_dict.update(
        {ii: v}
    )



Calculate the paired RMSD matrix between these transition states. As seen in our previous comparison plot, configurations of molecules with an RMSD lower than "0.1" can be considered the same.

In [ ]:
n_ts = len(opt_ts_xyzs)
rmsd_mat = np.ones((n_ts, n_ts)) * -2.5
for ii in range(n_ts):
    for jj in range(ii+1, n_ts):
        try:
            rmsd_mat[ii, jj] = np.log10(
                pymatgen_rmsd(
                    opt_ts_xyzs[ii],
                    opt_ts_xyzs[jj],
                    ignore_chirality=True,
                )
            )
        except:
            print(ii, jj)
            pass
        rmsd_mat[jj, ii] = rmsd_mat[ii, jj]



Classify these generated transition states based on their structures using K-Means.

In [ ]:
# TL fix of multiple OpenMB libraries loaded, cf. https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md from error message.
!export MKL_THREADING_LAYER=TBB 

In [ ]:
from sklearn.cluster import KMeans

def reorder_matrix(matrix, n_clusters):
    # Apply K-means clustering to rows and columns
    row_clusters = KMeans(n_clusters=n_clusters).fit_predict(matrix)
    
    # Create a permutation to reorder rows and columns
    row_permutation = np.argsort(row_clusters)
    col_permutation = np.argsort(row_clusters)

    # Apply the permutation to the matrix
    reordered_matrix = matrix[row_permutation][:, col_permutation]

    return reordered_matrix, row_permutation, row_clusters


n = n_ts  # Number of overall transition states
n_clusters = 6  # The number of clusters in our K-Means

reordered_matrix, row_permutation, row_clusters = reorder_matrix(rmsd_mat, n_clusters)



Display the classification results. It can be clearly seen that many transition state structures are quite similar (the red squares on the diagonal represent one cluster).

Note: We have applied a log10 transformation to the RMSD values. "-2" corresponds to "0.01A", and "-1" corresponds to "0.1A".

In [ ]:
from IPython.display import HTML

fig = px.imshow(
    reordered_matrix, 
    color_continuous_scale="Oryel_r",
    range_color=[-2, -0.3],
)
fig.layout.font.update({"size": 18, "family": "Arial"})

fig.layout.update({"width": 650, "height": 500})
# HTML(fig.to_html())
fig.show()


Intuitively summarize which cluster each different transition state belongs to (id: '0' to '5').

Note: The cluster where `./demo/example-3/opt_ts/26.ts.opt.xyz` is located corresponds to the true transition state of the given {reactant, product}.

In [ ]:
import json

cluster_dict = {}
for ii, cluster in enumerate(row_clusters):
    cluster = str(cluster)
    if cluster not in cluster_dict:
        cluster_dict[cluster] = [ind_dict[ii]]
    else:
        cluster_dict[cluster] += [ind_dict[ii]]

cluster_dict = OrderedDict(sorted(cluster_dict.items()))
cluster_dict


It can be seen that the geometries of the transition states in the different clusters are completely different.

In [ ]:
draw_in_3dmol(
    open("./demo/example-3/generated/gen_36_ts.xyz", "r").read(),
    "xyz"
)

### Are the rest of the TS worthless?

Of course not. Any given transition state structure will correspond to a unique chemical reaction. However, in calculations, due to personal experience or computational methods, <span style="color: red;">**the chemical reactions that each person can explore are also limited**</span>.

This limitation is particularly fatal especially when studying unknown complex reactions. It can cause us to overlook some potentially possible reactions, leading to a misjudgment of the reaction mechanism, which in turn affects the idea of catalytic material design. [Dr. Qiyuan Zhao's article](https://www.nature.com/articles/s43588-021-00101-3) explores these phenomena in depth, if you are interested in you can dive into it.

In our case, OA-ReactDiff not only found the transition state of the reaction we wanted, but also explored five related "unintended" chemical reactions due to the stochastic process feature. This feature can <span style="color: orange;">**compensate for the existing chemistry-based intuitive reaction exploration framework**</span> 😄.


For example, the reaction we generated below is **an unintended new reaction**.

In [ ]:
draw_reaction("./demo/example-3/irc", prefix="cluster", idx=1)


## Give me some atoms, and I can generate all the reactions about them<a id ='exploration'></a>

In addition to searching for transition states of known reactions, OA-ReactDiff can directly and unconditionally <span style="color: orange;">**generate new chemical reactions and be used to explore reaction networks**</span>

Here, we hypothesize a beaker with only four atoms, C, N, O, H, and use OA-ReactDiff to explore the possible reactions of all four atoms.


### Randomly Generated Reactions

In [ ]:
xyz_path = "./demo/CNOH/"
n_samples = 128  # Total number of reactions generated
natm = 4  # Number of atoms in the reactant
fragments_nodes = [
    torch.tensor([natm] * n_samples, device=device),
    torch.tensor([natm] * n_samples, device=device),
    torch.tensor([natm] * n_samples, device=device),
]

conditions = torch.tensor([[0]] * n_samples, device=device)
h0 = assemble_sample_inputs(
    atoms=["C"] * 1 + ["O"] * 1 + ["N"] * 1 + ["H"] * 1,  # The atomic species of the reactants, in this case one each of CNOH
    device=device,
    n_samples=n_samples,
    frag_type=False,
)

It is highly recommended that you use a V100 GPU machine to perform this step (it takes 30 seconds to run). Otherwise, it may take around 10 minutes. you can make yourself a cup of 🍵 or do some push-ups.

In [ ]:
out_samples, out_masks = ddpm_trainer.ddpm.sample(
    n_samples=n_samples,
    fragments_nodes=fragments_nodes,
    conditions=conditions,
    return_frames=1,
    timesteps=None,
    h0=h0,
)

Save the generated result as xyz files.

In [ ]:
write_tmp_xyz(
    fragments_nodes, 
    out_samples[0], 
    idx=[0, 1, 2], 
    ex_ind=0,
    localpath=xyz_path,
)



Randomly draw a generated reaction.

In [ ]:
idx = 2
assert idx < n_samples
views = draw_reaction(xyz_path, idx)
views



### Analyzing reaction results



#### Finding unique non-repeating stable Molecules

In [ ]:
from glob import glob

from pymatgen.io.xyz import XYZ
from openbabel import pybel

from oa_reactdiff.analyze.rmsd import pymatgen_rmsd


def xyz_to_smiles(fname: str) -> str:
    """Convert molecules in xyz format to smiles format

    Args:
        fname (str): path to the xyz file.

    Returns:
        str: SMILES string.
    """
    mol = next(pybel.readfile("xyz", fname))
    smi = mol.write(format="can")
    return smi.split()[0].strip()


In [ ]:
# TL fix:
xyz_path = "demo/CNOH/" #"demo/example-3/generated/"

In [ ]:
xyzfiles = glob(f"{xyz_path}/gen*_react.xyz") + glob(f"{xyz_path}/gen*_prod.xyz")
xyz_converter = XYZ(mol=None)
mol = xyz_converter.from_file(xyzfiles[0]).molecule
unique_mols = {xyzfiles[0]: mol}
for _xyzfile in xyzfiles:
    _mol = xyz_converter.from_file(_xyzfile).molecule
    min_rmsd = 100
    for _, mol in unique_mols.items():
        rmsd = pymatgen_rmsd(mol, _mol, ignore_chirality=True, threshold=0.5)
        min_rmsd = min(min_rmsd, rmsd)
    if min_rmsd > 0.1:  # If the rmsd with an existing molecule is all greater than 0.1, then it is considered a new molecule
        unique_mols.update({_xyzfile: _mol})
        
len(unique_mols)


Only consider stable (i.e. non-radical, etc.) molecules here.

In [ ]:
unique_idx = []
unique_smiles = []
idx = 0
for file in unique_mols:
    smi = xyz_to_smiles(file)
    if smi not in unique_smiles and not "." in smi:
        unique_smiles.append(smi)
        unique_idx.append(idx)
    idx += 1
unique_idx, unique_smiles  # Unique molecular counterparts to reactive indexes and smiles



Everyone can list it by themselves and see if the molecules produced here cover all the CONH tetraatom stabilized molecules 😁

Now let's draw one of them.

In [ ]:
idx = np.random.choice(unique_idx, 1)[0]

draw_in_3dmol(open(list(unique_mols.keys())[idx], "r").read(), "xyz")



#### Unique Reaction Pathway

We only analyze reactions between stable molecules

In [ ]:
unique_paths = {}
path_index = {}
for ii in range(n_samples):
    r_xyz = f"{xyz_path}/gen_{ii}_react.xyz"
    p_xyz = f"{xyz_path}/gen_{ii}_prod.xyz"
    path = set([xyz_to_smiles(r_xyz), xyz_to_smiles(p_xyz)])
    use = True
    for smi in path:
        if smi not in unique_smiles:
            use = False
    if not path in unique_paths.values() and len(path) > 1 and use:
        unique_paths[ii] = path
    if not (len(path) > 1 and use):
        continue
    sorted_smi = " & ".join(list(sorted(path)))
    if sorted_smi not in path_index:
        path_index[sorted_smi] = [ii]
    else:
        path_index[sorted_smi] += [ii]
mols_in_paths = []
for k, v in unique_paths.items():
    for _v in v:
        if not _v in mols_in_paths:
            mols_in_paths.append(_v)
            
mols_in_paths, len(mols_in_paths)


All CNOH tetraatomic stabilization reactions are as follows:

In [ ]:
unique_paths, len(unique_paths)


select one and see if this reaction is chemically reasonable 😄

In [ ]:
idx = np.random.choice(list(unique_paths.keys()), 4)[0] # 1, 2lm, 4 
views = draw_reaction(xyz_path, idx)
views



## Outlook: Let's "Generate" the Future <a id ='outlook'></a>
<div class="alert alert-success">
    <b>🎖️Congratulations! You have finished reading this notebook.</b>
</div>
    
A major challenge in the discovery of chemical materials lies in its enormous ($10^{60}$) potential design space. This space is so large that even if we were to buy all NVIDIA graphics cards and exhaust the global power supply, it would still be impossible to explore. Generative AI <span style="color: orange;">**bypasses the difficulty of screening the material space**</span> and directly generates potentially valuable molecules and materials, bringing new possibilities for material discovery.

Expansion of diffusion generation modeling for chemical materials is still in its early stages of infancy. If you have ideas about applying OA-ReactDiff to your own problems but are unsure if it is completely suitable or don't know where to start, please feel free to contact the author at duanchenru@gmail.com👏🏻



In [ ]:
# TL explorations:

In [ ]:
ProcessedTS1x